In [1]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *

In [2]:
PATH="/home/kirana/Documents/phd"
DATAPATH="/home/kirana/Documents/phd/data/experiment/AMAZON_BINARY"

## Dataset

In [ ]:
[df_train,df_valid,itos, train_tokens, valid_tokens, trn_lm, val_lm]=pickle.load(open(f'{DATAPATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [ ]:
bs=52 # 52 - Jeremey, 20 - default

In [ ]:
bptt= 70 #70 - Jeremey, 35 - default

In [ ]:
def create_data_lm(tokens,bs):
    import itertools
    # Collapse into a single large array
    tokens=np.asarray(list (itertools.chain(*tokens)))
    # How many batches
    n_batch=len(tokens)//bs
    # Truncate to exclude the ones at the end
    tokens=tokens[:bs*n_batch]
    # Reshape
    tokens=tokens.reshape(bs,-1)
    return tokens

In [ ]:
train_tokens=create_data_lm(df_train['tokens'],bs)
valid_tokens=create_data_lm(df_valid['tokens'],bs)
train_tokens.shape, valid_tokens.shape

In [ ]:
def get_batch(source, i):
    seq_len = min(args.bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

In [ ]:
n_batch=train_tokens.shape[1]
for i in range(0,n_batch,bptt):
    seq_len=min(bptt,n_batch-1-i)
    x=train_tokens[:,i:i+seq_len]
    y=train_tokens[:,i+1:i+1+seq_len]
    print (x.shape, y.shape)
    break

In [ ]:
x.shape,y.shape

In [ ]:
x,y

In [ ]:
x.shape,y.shape

In [ ]:
train_tokens.shape

## pre-trained AWD-LSTM

In [ ]:
PATH_AWD_LSTM='/home/kirana/Documents/phd/data/pre-trained/awd_lstm/'

In [ ]:
ls {PATH_AWD_LSTM}

In [ ]:
wgts = torch.load(f'{PATH_AWD_LSTM}/fwd_wt103.h5', map_location=lambda storage, loc: storage)

In [ ]:
wgts

In [ ]:
wgts = torch.load(f'{PATH_AWD_LSTM}/fwd_wt103_enc.h5', map_location=lambda storage, loc: storage)

In [ ]:
wgts

In [ ]:
itos2=pickle.load(open(f'{PATH_AWD_LSTM}/itos_wt103.pkl','rb'))
stoi2 = collections.defaultdict(lambda: -1, { v: k for k, v in enumerate(itos2) })

In [ ]:
wgts['encoder.weight'].shape

In [ ]:
enc_wgts = wgts['encoder.weight'].numpy() # converts np.ndarray from torch.FloatTensor.output shape: (238462, 400)
row_m = enc_wgts.mean(0) # returns the average of the array elements along axis 0. output shape: (400,)

In [ ]:
enc_wgts.shape, len(itos)

In [ ]:
n_emb=enc_wgts.shape[1]

In [ ]:
new_w = np.zeros((len(itos),n_emb), dtype=np.float32) # shape: (60002, 400)

for i, w in enumerate(itos):
    r = stoi2[w]
    new_w[i] = enc_wgts[r] if r >= 0 else row_m

In [ ]:
len(set(itos).difference(set(itos2)))

## Model Architecture

In [ ]:
from torch import nn
import torch
from torch.autograd import Variable

In [ ]:
n_inp=len(itos)
n_emb=400 #650
n_hidden=400 #650
n_layers=2
dropout=0.5
wd=1e-7

In [ ]:
class language_model (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,adaptive_log_softmax=True,tie_weights=True):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx
        self.adaptive_log_softmax,self.tie_weights=adaptive_log_softmax,tie_weights
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        self.gen_hidden()
        self.create_architecture()
        if pretrain_mtx is not None:
            print ("initializing")
            self.initialize_glove()
            
        if self.adaptive_log_softmax is False:
            self.criterion=nn.CrossEntropyLoss()
        
    def create_architecture(self):
        # Dropout layer
        self.dropout_enc=nn.Dropout(self.dropout_e)
        # Embedding Layer
        self.encoder=nn.Embedding(self.n_inp,self.n_emb)
        # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=False)
        self.dropout_op=nn.Dropout(self.dropout_o)
        
        if self.adaptive_log_softmax:
            # Adaptive Log Softmax Loss
            self.adaptive_softmax=AdaptiveLogSoftmaxWithLoss(self.n_hidden,
                                    self.n_inp,
                                    cutoffs=[round(self.n_inp/15),3*round(self.n_inp/15)],
                                    div_value=4,
                                    get_full_prob=True)
        else:
            self.decoder=nn.Linear(self.n_hidden,self.n_inp)
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
        if self.tie_weights:
            self.decoder.weight.requires_grad=False
        
    
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True
        if self.tie_weights:
            self.decoder.weight.requires_grad=True
        
    def initialize_glove(self):
        self.encoder.weight.data=torch.Tensor(self.pretrain_mtx)
        if self.tie_weights:
            self.decoder.weight=self.encoder.weight
    
    def gen_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    
        
    def forward(self,Xb,Yb):
        #import pdb
        #pdb.set_trace()
        embs=self.dropout_enc(self.encoder(Xb))
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())
        out,new_hidden=self.lstm(embs,self.hidden)
        out=self.dropout_op(out)
         # Wrap the hidden state in a new tensor without the gradients
        self.hidden=(Variable(new_hidden[0].data,requires_grad=False).to(self.device),\
                     Variable(new_hidden[1].data,requires_grad=False).to(self.device))
        if self.adaptive_log_softmax:
            out=out.reshape(out.size(0)*out.size(1),out.size(2))        # output is of shape n_batch * n_seq * n_hidden
      
            out=self.adaptive_softmax(out,Yb.view(-1))
            loss=out.loss
            preds=out.output_full
        else:
            #import pdb
            #pdb.set_trace()
            preds=self.decoder(out.contiguous().view(out.size(0)*out.size(1), out.size(2)))
            loss=self.criterion(preds,Yb.contiguous().view(-1))

        return preds, loss  

In [ ]:
def accuracy_multinomial(preds,actual):
    preds=preds.max(1)[1]
    correct=preds==actual
    return correct.float().sum()/len(correct)

In [ ]:
device="cuda:1"
model=language_model(n_inp,n_emb,n_hidden,n_layers,False,bs,device,0.05,0.5,0.5,new_w,False,True)


In [32]:
new_w.shape

(16206, 400)

In [33]:
model.encoder.weight.data.shape,model.decoder.weight.data.shape

(torch.Size([16206, 400]), torch.Size([16206, 400]))

In [34]:
nn.Linear(n_hidden,n_inp).weight.data.shape, n_hidden, n_inp

(torch.Size([16206, 400]), 400, 16206)

### Test if model forward works

In [35]:
model.decoder.weight.shape,model.encoder.weight.shape

(torch.Size([16206, 400]), torch.Size([16206, 400]))

In [36]:
x.shape

(52, 70)

In [37]:
if 1==0:
    model.forward(torch.LongTensor(x),torch.LongTensor(y))

In [38]:

model=model.to(device)

## Learner

In [39]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1

    
    def fit (self,Xb,Yb,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb)
        
       
            
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1))
            acc=acc.item()
            del preds
        
        if mode_train:
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 0.25)
            if 1==0:
                for p in self.model.parameters():
                    p.data.add_(self.lr, p.grad.data)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        n_batch=iterator.shape[1]
        epoch_loss,epoch_acc,i,k=0,0,0,0
        self.model.gen_hidden()
        #for k,i in enumerate(range(0,n_batch,self.bptt)):
        n_batch=iterator.shape[1]
        while i<n_batch-bptt:
            if mode_train:
                cust_bptt=self.bptt if np.random.random() < 0.95 else self.bptt//np.random.randint (2,4)
            else:
                cust_bptt=bptt
            seq_len=min(cust_bptt,n_batch-1-i)
            Xb=train_tokens[:,i:i+seq_len]
            Yb=train_tokens[:,i+1:i+1+seq_len]
            Xb=torch.LongTensor(Xb)
            Yb=torch.LongTensor(Yb)
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
            k=k+1
            i=i+seq_len
        epoch_loss=epoch_loss/k
        epoch_acc=epoch_acc/k
        
        if 1==0:
            lr /= 4.0
            # Freeze all the layers initially
            for param in resnet.parameters():
                param.requires_grad=False
            torch.save(resnet,'resnet')
            torch.save(resnet.state_dict(),'resnet_state_dict')
            resnet.load_state_dict(torch.load('resnet_state_dict'))
            for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr  
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

    
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        for epoch in range(n_epochs):
            
            loss,acc=self.run_epoch(dltrain,True)
            print (f'Epoch:{epoch} Loss:{loss}')
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Loss:{loss} Accuracy:{acc} Loss:{lossv} Accuracy:{accv}')
        

In [40]:
optimizer=torch.optim.Adam(model.parameters(),lr=1e-3,betas=(0.9,0.999), weight_decay=wd)
metric_fn=accuracy_multinomial

In [41]:
n_batch=np.int(np.ceil(train_tokens.shape[1]/bptt))
n_batch

442

In [42]:
model.freeze_embedding()

In [43]:
model.encoder.weight.requires_grad, model.decoder.weight.requires_grad

(False, False)

In [44]:
learner=Learner(model,optimizer,accuracy_multinomial,device,bptt,500,0.25)

In [45]:
model.n_inp

16206

In [46]:
learner.run_epochs(train_tokens,valid_tokens,5)

Epoch:0 Loss:6.942270007808651
Epoch:0 Loss:6.942270007808651 Accuracy:0.13707779799340772 Loss:6.303294668594996 Accuracy:0.17152015554408231
Epoch:1 Loss:6.353238028420343
Epoch:1 Loss:6.353238028420343 Accuracy:0.16178938607374826 Loss:5.893028477827708 Accuracy:0.18747139008094868
Epoch:2 Loss:6.006961266726058
Epoch:2 Loss:6.006961266726058 Accuracy:0.1728815727149677 Loss:5.541946709156036 Accuracy:0.2012534411624074
Epoch:3 Loss:5.705152110596368
Epoch:3 Loss:5.705152110596368 Accuracy:0.1820674755149937 Loss:5.222893178462982 Accuracy:0.21514424216002226
Epoch:4 Loss:5.429727396078869
Epoch:4 Loss:5.429727396078869 Accuracy:0.19062959191809714 Loss:4.93864827354749 Accuracy:0.22814790656169256


In [47]:
learner.run_epochs(train_tokens,valid_tokens,25)

Epoch:0 Loss:5.181079887710841
Epoch:0 Loss:5.181079887710841 Accuracy:0.20270939214171563 Loss:4.657285292943318 Accuracy:0.25449291585634154
Epoch:1 Loss:4.953199159279631
Epoch:1 Loss:4.953199159279631 Accuracy:0.24161001858187886 Loss:4.405627350012462 Accuracy:0.3194711667795976
Epoch:2 Loss:4.7511257588605345
Epoch:2 Loss:4.7511257588605345 Accuracy:0.2679013883719197 Loss:4.196161488691966 Accuracy:0.32993362347284955
Epoch:3 Loss:4.5757434692299155
Epoch:3 Loss:4.5757434692299155 Accuracy:0.2771412340695398 Loss:4.011842275659244 Accuracy:0.3403331196556489
Epoch:4 Loss:4.434489325804857
Epoch:4 Loss:4.434489325804857 Accuracy:0.2837197646039173 Loss:3.881600002447764 Accuracy:0.3468521187702815
Epoch:5 Loss:4.324775397514982
Epoch:5 Loss:4.324775397514982 Accuracy:0.2887851878803732 Loss:3.779797285795212 Accuracy:0.35254122627278167
Epoch:6 Loss:4.24448635785476
Epoch:6 Loss:4.24448635785476 Accuracy:0.29169240613346514 Loss:3.7049791564544043 Accuracy:0.357091361656785
Epoch

In [ ]:
learner.run_epochs(train_tokens,valid_tokens,25)

Epoch:0 Loss:3.873537317841454
Epoch:0 Loss:3.873537317841454 Accuracy:0.3181768861492123 Loss:3.2588779081900916 Accuracy:0.41362753448386985
Epoch:1 Loss:3.868222590365441
Epoch:1 Loss:3.868222590365441 Accuracy:0.31397012122835727 Loss:3.241112286845843 Accuracy:0.38897666707634926
Epoch:2 Loss:3.860879651861253
Epoch:2 Loss:3.860879651861253 Accuracy:0.3072229448486777 Loss:3.2422984590133033 Accuracy:0.38901673071086407
Epoch:3 Loss:3.851597041155385
Epoch:3 Loss:3.851597041155385 Accuracy:0.3078675010958604 Loss:3.2296492556730905 Accuracy:0.39083678523699444
Epoch:4 Loss:3.8460848053121355
Epoch:4 Loss:3.8460848053121355 Accuracy:0.32547321955012853 Loss:3.2283828804890313 Accuracy:0.4186756145209074
Epoch:5 Loss:3.8411132884078203
Epoch:5 Loss:3.8411132884078203 Accuracy:0.3099154373404732 Loss:3.2195165107647576 Accuracy:0.42012364603579044
Epoch:6 Loss:3.8350599329722557
Epoch:6 Loss:3.8350599329722557 Accuracy:0.324629207974986 Loss:3.20609421034654 Accuracy:0.42247025730709

In [48]:
torch.save(model.state_dict(),f'{DATAPATH}/inter/varybptt_model_state_dict')
torch.save(optimizer.state_dict(),f'{DATAPATH}/inter/varybptt_learner_state_dict')

In [49]:
model.unfreeze_embedding()

In [50]:
model.encoder.weight.requires_grad, model.decoder.weight.requires_grad

(True, True)

In [51]:
learner.run_epochs(train_tokens,valid_tokens,50)

Epoch:0 Loss:2.455076994214739
Epoch:0 Loss:2.455076994214739 Accuracy:0.4894116648605892 Loss:1.574017842610677 Accuracy:0.6905677914619446
Epoch:1 Loss:2.362198153057614
Epoch:1 Loss:2.362198153057614 Accuracy:0.5339623262753358 Loss:1.4725878238677979 Accuracy:0.7192308108011881
Epoch:2 Loss:2.2712207356014766
Epoch:2 Loss:2.2712207356014766 Accuracy:0.5521716805728706 Loss:1.3900961875915527 Accuracy:0.7353480259577433
Epoch:3 Loss:2.1909215859004427
Epoch:3 Loss:2.1909215859004427 Accuracy:0.5655832631247384 Loss:1.3382569551467896 Accuracy:0.7434982061386108
Epoch:4 Loss:2.1503394874366553
Epoch:4 Loss:2.1503394874366553 Accuracy:0.5730291685542545 Loss:1.2933137814203899 Accuracy:0.747619092464447
Epoch:5 Loss:2.1151228646437326
Epoch:5 Loss:2.1151228646437326 Accuracy:0.5771593401829401 Loss:1.2713415225346882 Accuracy:0.7546703616778055
Epoch:6 Loss:2.0871008430208478
Epoch:6 Loss:2.0871008430208478 Accuracy:0.5828100732394628 Loss:1.2303366263707478 Accuracy:0.760348002115885

In [52]:
learner.run_epochs(train_tokens,valid_tokens,20)

Epoch:0 Loss:1.4005129079560976
Epoch:0 Loss:1.4005129079560976 Accuracy:0.6779703530105384 Loss:0.5738116105397543 Accuracy:0.8760989507039388
Epoch:1 Loss:1.384764916367001
Epoch:1 Loss:1.384764916367001 Accuracy:0.6801707049210867 Loss:0.5655213793118795 Accuracy:0.8769231041272482
Epoch:2 Loss:1.3963093314852033
Epoch:2 Loss:1.3963093314852033 Accuracy:0.6796232666288103 Loss:0.5642988483111063 Accuracy:0.8788461883862814
Epoch:3 Loss:1.3806347846984863
Epoch:3 Loss:1.3806347846984863 Accuracy:0.6823077235903059 Loss:0.551474134127299 Accuracy:0.8780220150947571
Epoch:4 Loss:1.3543897458485195
Epoch:4 Loss:1.3543897458485195 Accuracy:0.6872370788029262 Loss:0.5427469611167908 Accuracy:0.8802197972933451
Epoch:5 Loss:1.3410670586994715
Epoch:5 Loss:1.3410670586994715 Accuracy:0.6885008113724845 Loss:0.5266106923421224 Accuracy:0.8869963884353638
Epoch:6 Loss:1.3506292036601475
Epoch:6 Loss:1.3506292036601475 Accuracy:0.6883438331740243 Loss:0.5227194825808207 Accuracy:0.887271086374

In [53]:
learner.run_epochs(train_tokens,valid_tokens,20)

Epoch:0 Loss:1.2286284617015293
Epoch:0 Loss:1.2286284617015293 Accuracy:0.7059439931597028 Loss:0.4248349368572235 Accuracy:0.906501849492391
Epoch:1 Loss:1.228726397241865
Epoch:1 Loss:1.228726397241865 Accuracy:0.7059327346937997 Loss:0.42250380913416546 Accuracy:0.9094322721163431
Epoch:2 Loss:1.243957143080862
Epoch:2 Loss:1.243957143080862 Accuracy:0.7039552519195958 Loss:0.4171498119831085 Accuracy:0.9095238447189331
Epoch:3 Loss:1.2298704896654402
Epoch:3 Loss:1.2298704896654402 Accuracy:0.7064207536833627 Loss:0.407338410615921 Accuracy:0.9103480180104574
Epoch:4 Loss:1.2144546045197382
Epoch:4 Loss:1.2144546045197382 Accuracy:0.7080510126219856 Loss:0.4001307984193166 Accuracy:0.9136447509129842
Epoch:5 Loss:1.2084093425008986
Epoch:5 Loss:1.2084093425008986 Accuracy:0.7102856387694677 Loss:0.40115875999132794 Accuracy:0.9130952755610148
Epoch:6 Loss:1.206751935822623
Epoch:6 Loss:1.206751935822623 Accuracy:0.7094348788261413 Loss:0.389671911795934 Accuracy:0.9185897707939148

In [54]:
torch.save(model.state_dict(),f'{DATAPATH}/inter/varybptt_model_state_dict_unfreeze')
torch.save(optimizer.state_dict(),f'{DATAPATH}/inter/varybptt_learner_state_dict_unfreeze')

In [55]:
torch.save (model,f'{DATAPATH}/inter/varybptt_model_awd_lstm')
torch.save (optimizer,f'{DATAPATH}/inter/varybptt_optimizer_awd_lstm')
torch.save (learner,f'{DATAPATH}/inter/varybptt_learner_awd_lstm')

/home/kirana/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type language_model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [56]:
pretrained_lm_weights=model.encoder.weight.data.cpu().numpy()
import pickle
pickle.dump(pretrained_lm_weights,open(f'{DATAPATH}/inter/varybpttpretrained_lm_weights','wb'))